In [1]:
# Import the libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
import sys
import string
from sklearn import datasets, linear_model
from aera import session
import os
pd.set_option('max_colwidth', 800)
import warnings
from loguru import logger
import math
import time
#warnings.filterwarnings('ignore')
from ipynb.fs.full import Utility
#import Utility

In [2]:
def readExternalDataset(dataset_name, col_list, dwb=None):
    """
    Read external dataset from Aera 
    """
    
    logger.info("reading dataset " + dataset_name + "...")
    data = dwb.read_report(reportName=dataset_name)
    #col_names = pd.DataFrame(data["read_dataset"]['data']['fields']).loc[:, "name"].tolist()
    col_names = col_list
            
    df = pd.DataFrame(data['data'])
    df, df.columns = df[1:], col_names
        
    return df

In [3]:
def readFinalReport(report_final, dwb=None):
    """
    Read final filtered/cleaned report from Aera with in scope data
    """
    
    logger.info("reading report " + report_final + "...")
    data = dwb.read_report(reportName=report_final)
    #col_names = pd.DataFrame(data["read_dataset"]['data']['fields']).loc[:, "name"].tolist()
    col_names = ["ForecastUnitCode", "PlantCode", "EccCustomerLevel7Code", "ActualDeliveryDate", 
                 "Category", "Model_Hierarchy", "ActualDeliveredQuantity", "Update_Date", "Insert_Date"]
                
    df = pd.DataFrame(data['data'])
    df, df.columns = df[1:], col_names
        
    return df

In [4]:
def preProcessFinalReport(df_pre):
    """
    Pre-process final filtered/cleaned dataset
    """
    
    logger.info("pre-processing dataset_subprocess_last...")
    df = df_pre.copy()
    df = df[df["ActualDeliveryDate"] != "XXXX"]
    df["ActualDeliveryDate"] = df["ActualDeliveryDate"].astype("str")
    df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
    df["ActualDeliveredQuantity"] = df["ActualDeliveredQuantity"].astype("float")
    df["Year"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.year
    df = df[df["Year"] != "1999"]
    df = df[df["Year"] != 1999]
    df = df[df["Model_Hierarchy"].notna()]
    df = df[["ForecastUnitCode", "EccCustomerLevel7Code", "PlantCode", "Model_Hierarchy", 
             "ActualDeliveryDate", "ActualDeliveredQuantity"]]
                
    return df

In [5]:
def preProcessandEnrichment(final_df, startDate, endDate,
                            covid_stringency, dc_demo_data, google_covid_mobility, holiday_info):
    """
    Pre-process and enrichment of primary sales data
    """
    
    logger.info("pre-processing/enriching of primary sales data...")
    logger.info("pre-processing started at..." + str(datetime.now().time()))
    listOfHierarchy = final_df["Model_Hierarchy"].unique()
    dataModel = pd.DataFrame()
    #listOfHierarchy = ["00011111123397_0030026728_3990"]
    df_inPandas = final_df.copy()
    df_typeCasted = Utility.dataPreprocessingPrimarySales.typeCast(df_inPandas)
    df_inPandas = df_inPandas.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
    df_missingDayFilled = df_inPandas.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.dataPreprocessingPrimarySales.missingDayFill(grp, startDate, endDate)).reset_index(drop=True)
    df_reTypeCasted = Utility.dataPreprocessingPrimarySales.typeCast(df_missingDayFilled)
    df_initialZeroRemoved = df_reTypeCasted.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.dataPreprocessingPrimarySales.initialZeroRemoval(grp)).reset_index(drop=True)
    df_MOQLotSize = df_initialZeroRemoved.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.dataPreprocessingPrimarySales.getMOQAndLotSize(grp)).reset_index(drop=True)
    df_primarySalesPreProcessed = df_MOQLotSize.copy()
    df_primarySalesPreProcessed = df_MOQLotSize.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
    logger.info("initial pre-processing completed at..." + str(datetime.now().time()))
    
    df_withDC = Utility.enrichmentWithExternalData.getDCDemographicInfo(df_primarySalesPreProcessed, dc_demo_data)
    df_withStringency = Utility.enrichmentWithExternalData.getCovidStringency(df_withDC, covid_stringency)
    df_withMobility = Utility.enrichmentWithExternalData.getGoogleMobility(df_withStringency, google_covid_mobility)
    df_withHoliday = Utility.enrichmentWithExternalData.getHolidayInfo(df_withMobility, holiday_info)
    df_withWeather = Utility.enrichmentWithExternalData.getWeatherInfo(df_withHoliday)

    df_primarySalesEnrichedExternal = df_withWeather.copy()
    logger.info("enrichment with external data completed at..." + str(datetime.now().time()))
    df_primarySalesEnrichedExternal = df_primarySalesEnrichedExternal.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
    df_primarySalesEnrichedExternal = df_primarySalesEnrichedExternal[["ForecastUnitCode", "EccCustomerLevel7Code", 
                                                                       "Model_Hierarchy", "ActualDeliveryDate", 
                                                                       "ActualDeliveredQuantity", "PlantCode", 
                                                                       "MOQ", "Lot_Size", "StringencyIndex", "Ctr", 
                                                                       "retail_and_recreation_percent_change_from_baseline", 
                                                                       "grocery_and_pharmacy_percent_change_from_baseline", "isHoliday", 
                                                                       "Holiday_Desc", "Season"]]
    
    df_missingValueFilled = Utility.dataAdditionalprocessing.fillMissingValue(df_primarySalesEnrichedExternal)
    df_holidayDescEdited = df_missingValueFilled.copy() 
    df_holidayDescEdited["Holiday_Desc"] = df_holidayDescEdited["Holiday_Desc"].apply(Utility.dataAdditionalprocessing.formatHolidayDesc)

    logger.info("2nd phase of pre-processing completed at..." + str(datetime.now().time()))
    df_withHolidayPeriod = Utility.dataAdditionalprocessing.identifyHolidayPeriod(df_holidayDescEdited, holiday_info)
    logger.info("holiday period identification completed at..." + str(datetime.now().time()))
    df_featureEngineered = Utility.dataAdditionalprocessing.featureEnggDate(df_withHolidayPeriod)
    logger.info("feature engg. (Date) completed at..." + str(datetime.now().time()))
    df_featureEngineered = df_featureEngineered.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.dataAdditionalprocessing.featureEnggLag(grp)).reset_index(drop=True)
    logger.info("feature engg. (Lag) completed at..." + str(datetime.now().time()))
    df_primarySalesCapped = df_featureEngineered.copy()
    df_typeCastFinal =  Utility.dataAdditionalprocessing.typeCast(df_primarySalesCapped)
    # Drop additional columns
    df_train = df_typeCastFinal.drop(["isHoliday", "Ctr"], axis=1)
    
    logger.info("train data processing completed at..." + str(datetime.now().time()))
    df_testA = df_train.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.futureDataGeneration.genFutureDataStepA(grp, endDate, dc_demo_data, holiday_info)).reset_index(drop=True)
    df_testB = Utility.dataAdditionalprocessing.identifyHolidayPeriod(df_testA, holiday_info)
    df_test = df_testB.groupby(by="Model_Hierarchy").apply(lambda grp: Utility.futureDataGeneration.genFutureDataStepC(grp)).reset_index(drop=True)
    
    logger.info("test data processing completed at..." + str(datetime.now().time()))    
    
    #dataModel = dataModel.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
    #print(df_train.tail())
    #print(df_train.shape)
    #print(df_test.shape)
    #print(df_test.tail())
    df_finalDataForModel = pd.concat([df_train, df_test], axis=0)

    dataModel = pd.concat([dataModel, df_finalDataForModel], ignore_index=True)
    dataModel["StringencyIndex"] = dataModel["StringencyIndex"].fillna(0.0)
    dataModel = dataModel.sort_values(by=["Model_Hierarchy", "ActualDeliveryDate"], ascending=True).reset_index(drop=True)    
    dataModel.to_csv("/home/cortex/Demand Sensing/OP/dataModel_Dressings.csv", index=False)
    logger.info("final pre-processing completed at..." + str(datetime.now().time()))
                         
        
    return dataModel

In [6]:
def createModelData(df):
    """
    Create data for ML model
    """
    
    logger.info("creating data for ML model...")
    dataModel_final = pd.get_dummies(df, columns=["Holiday_Desc", "Season", "isBeforeHoliday", 
                                                  "isAfterHoliday", "isWeekend", "isMonthStartDate", 
                                                  "isMonthEndDate", "isQtrStartDate", "isQtrEndDate"])
   
    dataModel_final.to_csv("/home/cortex/Demand Sensing/OP/ML_FINAL_INPUT_DT0714_Dressings.csv", index=False)
    return dataModel_final

In [7]:
def saveAeraDataset(df, dataset, dwb):
    """
    Save the dataset
    """
    
    logger.info("saving final data from ML model...")
    dataset_file = f'/tmp/{dataset}.csv'
    df.to_csv(dataset_file, index=False)
        
    def getFileSize(path):
        """
        Calculating the file size
        """
        
        size = os.path.getsize(path) / (1024 * 1024)
        return math.ceil(size)

    # splitting the csv into chunks; each chunk is to less than 18MB
    total_file_size = getFileSize(dataset_file)
    number_of_chunks = math.ceil(total_file_size / 18)
        
    df = pd.read_csv(dataset_file)
    create_dataset_response = None
    
    for idx, chunk in enumerate(np.array_split(df, number_of_chunks)):
        chunk_file_path=f'/tmp/chunk_file_{dataset}.csv'
        chunk.to_csv(chunk_file_path, index=False)
     
        create_dataset_response = dwb.create_dataset(workspaceName="Demand Sensing",
                                                     dataSetName=dataset,
                                                     description=dataset,
                                                     folderPath="Subprocess_ML_Res0715",
                                                     file=chunk_file_path,
                                                     isAppend=True
                                                    )
    
    return create_dataset_response

In [8]:
def main(req_data, sdk_session=None):
    """
    Main runner
    """
        
    # intializing aera session
    if not sdk_session:
        from aera import session 
        sdk_session = session.get_session()
    dwb = sdk_session.create_client('dwb')
    
    report_final = req_data['report_final'] 
    dataset_name_stringency = req_data['dataset_name_stringency'] 
    dataset_name_mobility = req_data['dataset_name_mobility'] 
    dataset_name_holiday = req_data['dataset_name_holiday'] 
    dataset_name_demo = req_data['dataset_name_demo'] 

    # read external datasets
    col_list_stringency = ['CountryName', 'Formatted_Date', 'RegionName', 'StringencyIndex'] 
    covid_stringency = readExternalDataset(dataset_name=dataset_name_stringency, 
                                           col_list=col_list_stringency, dwb=dwb)
    
    col_list_demo = ['City', 'County', 'Ctr', 'Full_Country_Name', 
                     'Full_State_Name', 'Place', 'Plnt', 'PostalCode', 'Rg']
    dc_demo_data = readExternalDataset(dataset_name=dataset_name_demo, 
                                       col_list=col_list_demo, dwb=dwb)
    
    col_list_mobility = ['country_region', 'country_region_code', 'date', 
                         'grocery_and_pharmacy_percent_change_from_baseline', 
                         'retail_and_recreation_percent_change_from_baseline', 
                         'sub_region_1', 'sub_region_2']
    google_covid_mobility = readExternalDataset(dataset_name=dataset_name_mobility, 
                                                col_list=col_list_mobility, dwb=dwb)
        
    col_list_holiday = ['Country', 'Date', 'Holiday Description', 'Jurisdiction']
    holiday_info = readExternalDataset(dataset_name=dataset_name_holiday, 
                                       col_list=col_list_holiday, dwb=dwb)
    
    # read datasets/reports
    df = readFinalReport(report_final=report_final, dwb=dwb)
       
    # pre-process datasets/reports
    processed_df = preProcessFinalReport(df)
              
    # pre-process and enrichment of primary sales
    startDate = processed_df["ActualDeliveryDate"].min()
    endDate = processed_df["ActualDeliveryDate"].max()
    dataModel_df = preProcessandEnrichment(processed_df, startDate, endDate, covid_stringency, 
                                           dc_demo_data, google_covid_mobility, holiday_info)
     
    # create final dataset for ML model
    dataModel_final = createModelData(dataModel_df)
    
    # saving and exporting new dataset
    output_dataset = "DS_ML_Dressings_RES0715"
    new_dataset_name = output_dataset   
    new_dataset = saveAeraDataset(dataModel_final, new_dataset_name, dwb)
    resp_data = {"new_dataset_name": new_dataset_name}
    
    return resp_data

In [9]:
# Define entry point and run
if __name__ == '__main__':
    from aera import session 
    sdk_session = session.get_session()
    dwb = sdk_session.create_client('dwb')

    req_data = {"report_final": "DemandSensing_Dressings",
                "dataset_name_stringency": "NA_Covid_Stringency_Data",
                "dataset_name_mobility": "NA_Covid_Mobility_Data",
                "dataset_name_holiday": "NA_Holiday_List",
                "dataset_name_demo": "NA_DC_Demographic_Data"}
    
    main(req_data, sdk_session=sdk_session)

2022-07-18 16:51:47.797 | INFO     | __main__:readExternalDataset:6 - reading dataset NA_Covid_Stringency_Data...


credentials from config file
You (subhamoy.bhaduri@unilever.com) are using project id (DB50B34F_0DDD_4D6B_914D_9677EED86C59)
Method execution status Success


2022-07-18 16:51:49.938 | INFO     | __main__:readExternalDataset:6 - reading dataset NA_DC_Demographic_Data...
2022-07-18 16:51:50.559 | INFO     | __main__:readExternalDataset:6 - reading dataset NA_Covid_Mobility_Data...


Method execution status Success
Method execution status Success


2022-07-18 16:52:16.497 | INFO     | __main__:readExternalDataset:6 - reading dataset NA_Holiday_List...
2022-07-18 16:52:17.554 | INFO     | __main__:readFinalReport:6 - reading report DemandSensing_All_1407...


Method execution status Success


2022-07-18 16:52:18.228 | INFO     | __main__:preProcessFinalReport:6 - pre-processing dataset_subprocess_last...
2022-07-18 16:52:18.257 | INFO     | __main__:preProcessandEnrichment:7 - pre-processing/enriching of primary sales data...
2022-07-18 16:52:18.257 | INFO     | __main__:preProcessandEnrichment:8 - pre-processing started at...16:52:18.257949


Method execution status Success


2022-07-18 16:52:18.834 | INFO     | __main__:preProcessandEnrichment:21 - initial pre-processing completed at...16:52:18.834699
2022-07-18 16:52:25.656 | INFO     | __main__:preProcessandEnrichment:30 - enrichment with external data completed at...16:52:25.656565
2022-07-18 16:52:25.757 | INFO     | __main__:preProcessandEnrichment:44 - 2nd phase of pre-processing completed at...16:52:25.757751
2022-07-18 16:52:25.857 | INFO     | __main__:preProcessandEnrichment:46 - holiday period identification completed at...16:52:25.857116
2022-07-18 16:52:25.967 | INFO     | __main__:preProcessandEnrichment:48 - feature engg. (Date) completed at...16:52:25.967717
2022-07-18 16:52:26.187 | INFO     | __main__:preProcessandEnrichment:50 - feature engg. (Lag) completed at...16:52:26.187891
2022-07-18 16:52:26.314 | INFO     | __main__:preProcessandEnrichment:56 - train data processing completed at...16:52:26.314103
2022-07-18 16:52:28.625 | INFO     | __main__:preProcessandEnrichment:61 - test data

In [10]:
"""
dm_dataset = "/home/cortex/Demand Sensing/External Variables/dataModel_final_with_dummy.csv"
new_dataset_dm = dwb.create_dataset(workspaceName="NA DRP",
                                    dataSetName="dataModel_withdummy",
                                    description="dataModel_withdummy",
                                    folderPath="Demand Sensing",
                                    file=dm_dataset
                                   )
"""

'\ndm_dataset = "/home/cortex/Demand Sensing/External Variables/dataModel_final_with_dummy.csv"\nnew_dataset_dm = dwb.create_dataset(workspaceName="NA DRP",\n                                    dataSetName="dataModel_withdummy",\n                                    description="dataModel_withdummy",\n                                    folderPath="Demand Sensing",\n                                    file=dm_dataset\n                                   )\n'